In [1]:
import pandas as pd
# df_item = pd.read_csv('./tianchi_fresh_comp_train_item.csv')
df_user = pd.read_csv('./tianchi_fresh_comp_train_user.csv')

根据日期范围11.18~12.18分割数据集：

- 抽取11.18~12.17的数据作为训练集
- 抽取12.18的数据作为测试集
- 制作标签：
- 其中11.18~12.17的数据集的标签用这段期间内是否曾经过买过来拟合买/不买

处理数据集。我们需要一个不含2014-12-17和2014-12-18的条目但有是否购买标签的训练集;
我们还需要一个不含2014-12-18的条目但有是否购买标签的测试集;
最后就是我们整一个数据集，没有标签，用作提交时的输入.

如何制作标签/如何根据特定条件新建一列： [HERE](https://stackoverflow.com/questions/19913659/pandas-conditional-creation-of-a-series-dataframe-column)

In [2]:
#删掉我用不上的地理哈希值
del df_user['user_geohash']

In [3]:
df_user.shape

(23291027, 5)

In [4]:
df_user.head()

,user_id,item_id,behavior_type,item_category,time
0,10001082,285259775,1,4076,2014-12-08 18
1,10001082,4368907,1,5503,2014-12-12 12
2,10001082,4368907,1,5503,2014-12-12 12
3,10001082,53616768,1,9762,2014-12-02 15
4,10001082,151466952,1,5232,2014-12-12 11


In [5]:
# 选出17号买的行/选出18号买的行
df_buy_17 = df_user.loc[df_user['time'].str.contains('2014-12-17')]
df_buy_18 = df_user.loc[df_user['time'].str.contains('2014-12-18')]

# 只保留user_id和item_id 之后用来join
df_buy_17 = df_buy_17[['user_id','item_id']]
df_buy_18 = df_buy_18[['user_id','item_id']]

# 去掉重复的user_id - item_id列。因为第二天用户可能在同一个子集买不止一件商品，所以会有重复
# 重复会影响merge的结果，所以merge前要去重
df_buy_17 = df_buy_17.drop_duplicates()
df_buy_18 = df_buy_18.drop_duplicates()

# 增加标签
df_buy_17['buy_17'] = 1
df_buy_18['buy_18'] = 1

# merge操作
df_user = pd.merge(left=df_user, right=df_buy_17, how='left', on=['user_id','item_id'])
df_user = pd.merge(left=df_user, right=df_buy_18, how='left', on=['user_id','item_id'])

In [6]:
# 把不在df_buy_17和df_buy_18的值都填为0
df_user[['buy_17','buy_18']] = df_user[['buy_17','buy_18']].fillna(0)
df_user.head()

,user_id,item_id,behavior_type,item_category,time,buy_17,buy_18
0,10001082,285259775,1,4076,2014-12-08 18,0.0,0.0
1,10001082,4368907,1,5503,2014-12-12 12,0.0,0.0
2,10001082,4368907,1,5503,2014-12-12 12,0.0,0.0
3,10001082,53616768,1,9762,2014-12-02 15,0.0,0.0
4,10001082,151466952,1,5232,2014-12-12 11,0.0,0.0


# regular definition
用人工规则创作特征：
脑洞越大，越贴近业务实际越好

- 商品总浏览次数
- 商品前一天，也就是18号有没有加入购物车：前一天加入购物车的最有可能被买
- 商品有没有曾经（包括前一天）被加购物车，曾经被加购物车的商品最有可能被买，即便不是前一天加进去的
- 商品有没有曾经被收藏：有收藏的最有可能被买
- 商品总浏览次数：浏览的越多越有可能被买
- 该用户总购买次数：用户购买东西越多，用户购买力越强
- 该商品被购买次数：商品被买的越多，该商品越畅销

训练集：

| 用户商品id|总收藏次数|在购物车次数|16号加购物车数量|总浏览次数|用户购买次数|商品被买次数|17号是否购买|
|---|---|---|---|---|---|
|10001082-4076|5|2|1|5|20|30|否

测试集

| 用户商品id|总收藏次数|在购物车次数|17号加购物车数量|总浏览次数|用户购买次数|商品被买次数|18号是否购买|
|---|---|---|---|---|---|
|10001082-4076|5|2|1|5|20|30|是

要提交的最终模型训练集:

|用户商品id|总收藏次数|在购物车次数|18号加购物车数量|总浏览次数|用户购买次数|商品被买次数|19号是否购买|
|---|---|---|---|---|---|
|10001082-4076|5|2|1|5|20|30|待预测

最后根据得到的预测结果筛选出用户商品id和购买为**是**的列，整理成用户-商品id的对

具体思路：

- 用户购买数:用`groupby` 将用户id作为一个键，然后`count(behavior_type)` 然后再取behavior_type=4的count值

- 商品被购买次数:用`groupby` 将商品id作为一个键，然后`count(behavior_type)` 然后再取behavior_type=4的count值

- 总浏览次数：用`groupby`将用户id和商品id作为一个键。然后`count(behavior_type)` 然后再取behavior_type=1的count值

- 总收藏次数：用`groupby`将用户id和商品id作为一个键。然后`count(behavior_type)` 然后再取behavior_type=2的count值

- 在购物车次数：用`groupby`将用户id和商品id作为一个键。然后`count(behavior_type)` 然后再取behavior_type=3的count值

- 前一天在购物车次数：用`groupby`将用户id和商品id作为一个键。然后`count(behavior_type)` 然后再取behavior_type=3 且 time = 前一天日期 的count值

关于groupby和count可以看[这里](https://stackoverflow.com/questions/10373660/converting-a-pandas-groupby-object-to-dataframe)

In [7]:
# 用户购买次数
df_user_behavior_count = pd.DataFrame({'count_user_buy' : df_user[['user_id','behavior_type']].groupby(['user_id','behavior_type']).size()}).reset_index()
# 取出behavior_type=4的行
df_user_buy_count = df_user_behavior_count.loc[df_user_behavior_count['behavior_type'] == 4]
# 删掉behavior_type以便用来join
del df_user_buy_count['behavior_type']
#与原来的user表join在一起:
# eg: caller.join(other.set_index('key'), on='key')
df_user = df_user.join(df_user_buy_count.set_index('user_id'), on='user_id')
# 删掉这两个临时数据表
del df_user_buy_count
del df_user_behavior_count
df_user.head()

,user_id,item_id,behavior_type,item_category,time,buy_17,buy_18,count_user_buy
0,10001082,285259775,1,4076,2014-12-08 18,0.0,0.0,4.0
1,10001082,4368907,1,5503,2014-12-12 12,0.0,0.0,4.0
2,10001082,4368907,1,5503,2014-12-12 12,0.0,0.0,4.0
3,10001082,53616768,1,9762,2014-12-02 15,0.0,0.0,4.0
4,10001082,151466952,1,5232,2014-12-12 11,0.0,0.0,4.0


In [8]:
df_user.shape

(23291027, 8)

In [9]:
# 商品被购买次数:
df_item_behavior_count = pd.DataFrame({'count_item_bought' : df_user[['item_id','behavior_type']].groupby(['item_id','behavior_type']).size()}).reset_index()
df_item_bought_count =  df_item_behavior_count.loc[df_item_behavior_count['behavior_type'] == 4]
del df_item_bought_count['behavior_type']
df_user = df_user.join(df_item_bought_count.set_index('item_id'), on='item_id')
del df_item_bought_count
del df_item_behavior_count
df_user = df_user.fillna(0)
df_user.head()

,user_id,item_id,behavior_type,item_category,time,buy_17,buy_18,count_user_buy,count_item_bought
0,10001082,285259775,1,4076,2014-12-08 18,0.0,0.0,4.0,0.0
1,10001082,4368907,1,5503,2014-12-12 12,0.0,0.0,4.0,0.0
2,10001082,4368907,1,5503,2014-12-12 12,0.0,0.0,4.0,0.0
3,10001082,53616768,1,9762,2014-12-02 15,0.0,0.0,4.0,2.0
4,10001082,151466952,1,5232,2014-12-12 11,0.0,0.0,4.0,0.0


In [10]:
df_user.shape

(23291027, 9)

In [11]:
# 获取某一用户对某一商品的操作的count
df_user_item_behavior_count = pd.DataFrame({'count_behavior' : df_user[['user_id','item_id','behavior_type']].groupby(['user_id','item_id','behavior_type']).size()}).reset_index()

In [12]:
df_user_item_behavior_count

,user_id,item_id,behavior_type,count_behavior
0,492,254885,1,9
1,492,254885,3,1
2,492,254885,4,1
3,492,2316002,1,3
4,492,3473697,1,2
5,492,6983065,1,2
6,492,8977055,1,2
7,492,9695336,1,2
8,492,14259295,1,2
9,492,30541025,1,2


In [13]:
# 某个商品被某个用户浏览的总次数
df_user_item_look_count =  df_user_item_behavior_count.loc[df_user_item_behavior_count['behavior_type'] == 1]
df_user_item_look_count = df_user_item_look_count.rename(columns = {'count_behavior':'count_look'})
del df_user_item_look_count['behavior_type']

In [14]:
# 某个商品被某个用户收藏的总次数
df_user_item_favor_count =  df_user_item_behavior_count.loc[df_user_item_behavior_count['behavior_type'] == 2]
df_user_item_favor_count = df_user_item_favor_count.rename(columns = {'count_behavior':'count_favor'})
del df_user_item_favor_count['behavior_type']

In [15]:
# 某个商品被某个用户添加购物车的次数
df_user_item_addcart_count =  df_user_item_behavior_count.loc[df_user_item_behavior_count['behavior_type'] == 3]
df_user_item_addcart_count = df_user_item_addcart_count.rename(columns = {'count_behavior':'count_addcart'})
del df_user_item_addcart_count['behavior_type']

In [16]:
# 某个商品被某个用户在16号添加进购物车的次数
df_user_16 = df_user.loc[df_user['time'].str.contains('2014-12-16')]
df_user_item_behavior_16_count = pd.DataFrame({'count_behavior' : df_user_16[['user_id','item_id','behavior_type']].groupby(['user_id','item_id','behavior_type']).size()}).reset_index()
df_user_item_addcart_16_count = df_user_item_behavior_16_count.loc[df_user_item_behavior_16_count['behavior_type'] == 3]
df_user_item_addcart_16_count = df_user_item_addcart_16_count.rename(columns = {'count_behavior':'count_addcart_16'})
del df_user_item_addcart_16_count['behavior_type']

In [17]:
# 某个商品被某个用户在17号添加进购物车的次数
df_user_17 = df_user.loc[df_user['time'].str.contains('2014-12-17')]
df_user_item_behavior_17_count = pd.DataFrame({'count_behavior' : df_user_17[['user_id','item_id','behavior_type']].groupby(['user_id','item_id','behavior_type']).size()}).reset_index()
df_user_item_addcart_17_count = df_user_item_behavior_17_count.loc[df_user_item_behavior_17_count['behavior_type'] == 3]
df_user_item_addcart_17_count = df_user_item_addcart_17_count.rename(columns = {'count_behavior':'count_addcart_17'})
del df_user_item_addcart_17_count['behavior_type']

In [18]:
# 某个商品被某个用户在18号添加进购物车的次数
df_user_18 = df_user.loc[df_user['time'].str.contains('2014-12-18')]
df_user_item_behavior_18_count = pd.DataFrame({'count_behavior' : df_user_18[['user_id','item_id','behavior_type']].groupby(['user_id','item_id','behavior_type']).size()}).reset_index()
df_user_item_addcart_18_count = df_user_item_behavior_18_count.loc[df_user_item_behavior_18_count['behavior_type'] == 3]
df_user_item_addcart_18_count = df_user_item_addcart_18_count.rename(columns = {'count_behavior':'count_addcart_18'})
del df_user_item_addcart_18_count['behavior_type']

In [19]:
df_user.head()

,user_id,item_id,behavior_type,item_category,time,buy_17,buy_18,count_user_buy,count_item_bought
0,10001082,285259775,1,4076,2014-12-08 18,0.0,0.0,4.0,0.0
1,10001082,4368907,1,5503,2014-12-12 12,0.0,0.0,4.0,0.0
2,10001082,4368907,1,5503,2014-12-12 12,0.0,0.0,4.0,0.0
3,10001082,53616768,1,9762,2014-12-02 15,0.0,0.0,4.0,2.0
4,10001082,151466952,1,5232,2014-12-12 11,0.0,0.0,4.0,0.0


In [20]:
del df_user['item_category']
del df_user['behavior_type']
df_user = df_user.drop_duplicates()

In [21]:
df_user.shape

(10654899, 7)

In [22]:
df_user.head()

,user_id,item_id,time,buy_17,buy_18,count_user_buy,count_item_bought
0,10001082,285259775,2014-12-08 18,0.0,0.0,4.0,0.0
1,10001082,4368907,2014-12-12 12,0.0,0.0,4.0,0.0
3,10001082,53616768,2014-12-02 15,0.0,0.0,4.0,2.0
4,10001082,151466952,2014-12-12 11,0.0,0.0,4.0,0.0
6,10001082,290088061,2014-12-12 12,0.0,0.0,4.0,0.0


In [23]:
df_user = pd.merge(df_user, df_user_item_look_count, how='left', left_on=['user_id','item_id'], right_on=['user_id','item_id'])
df_user = pd.merge(df_user, df_user_item_favor_count, how='left', left_on=['user_id','item_id'], right_on=['user_id','item_id'])
df_user = pd.merge(df_user, df_user_item_addcart_count, how='left', left_on=['user_id','item_id'], right_on=['user_id','item_id'])
df_user = pd.merge(df_user, df_user_item_addcart_16_count, how='left', left_on=['user_id','item_id'], right_on=['user_id','item_id'])
df_user = pd.merge(df_user, df_user_item_addcart_17_count, how='left', left_on=['user_id','item_id'], right_on=['user_id','item_id'])
df_user = pd.merge(df_user, df_user_item_addcart_18_count, how='left', left_on=['user_id','item_id'], right_on=['user_id','item_id'])

In [24]:
df_user = df_user.fillna(0)
df_user.head()

,user_id,item_id,time,buy_17,buy_18,count_user_buy,count_item_bought,count_look,count_favor,count_addcart,count_addcart_16,count_addcart_17,count_addcart_18
0,10001082,285259775,2014-12-08 18,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,10001082,4368907,2014-12-12 12,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2,10001082,53616768,2014-12-02 15,0.0,0.0,4.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0
3,10001082,151466952,2014-12-12 11,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
4,10001082,290088061,2014-12-12 12,0.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0


# create dataset

In [25]:
# 提取不含17，18号的数据
# 因为标签只是buy_17所以删掉buy_18
# 因为是预测17号的，所以只需要16号加购物车的数据，删掉17号加购物车的数据
df_user_train = df_user.loc[~df_user['time'].str.contains('(2014-12-17*)|(2014-12-18*)')]
del df_user_train['time']
del df_user_train['buy_18']
del df_user_train['count_addcart_17']
del df_user_train['count_addcart_18']

# 提取不含18号的数据
# 因为标签只是buy_18所以删掉buy_17
# 因为是预测18号的，所以只需要17号加购物车的数据，删掉16号加购物车的数据
df_user_test = df_user.loc[~df_user['time'].str.contains('2014-12-18')]
del df_user_test['time']
del df_user_test['buy_17']
del df_user_test['count_addcart_16']
del df_user_test['count_addcart_18']


# 采用所有的数据
# 因为是提交的所以没有标签
df_user_submit = df_user.copy()
del df_user_submit['time']
del df_user_submit['buy_18']
del df_user_submit['buy_17']
del df_user_submit['count_addcart_16']
del df_user_submit['count_addcart_17']

print(df_user_train.head())
print('------------------')
print(df_user_test.head())
print('------------------')
print(df_user_submit.head())

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


     user_id    item_id  buy_17  count_user_buy  count_item_bought  \
0   10001082  285259775     0.0             4.0                0.0   
2   10001082   53616768     0.0             4.0                2.0   
9   10001082    9947871     0.0             4.0                0.0   
11  10001082  275221686     1.0             4.0                3.0   
12  10001082   97441652     0.0             4.0                0.0   

    count_look  count_favor  count_addcart  count_addcart_16  
0          1.0          0.0            0.0               0.0  
2          5.0          0.0            0.0               0.0  
9          1.0          0.0            0.0               0.0  
11        30.0          0.0            0.0               0.0  
12         1.0          0.0            0.0               0.0  
------------------
    user_id    item_id  buy_18  count_user_buy  count_item_bought  count_look  \
0  10001082  285259775     0.0             4.0                0.0         1.0   
1  10001082    43689

In [26]:
df_user_train = df_user_train.drop_duplicates()
df_user_test = df_user_test.drop_duplicates()
df_user_submit = df_user_submit.drop_duplicates()

In [27]:
df_user_train.head()

,user_id,item_id,buy_17,count_user_buy,count_item_bought,count_look,count_favor,count_addcart,count_addcart_16
0,10001082,285259775,0.0,4.0,0.0,1.0,0.0,0.0,0.0
2,10001082,53616768,0.0,4.0,2.0,5.0,0.0,0.0,0.0
9,10001082,9947871,0.0,4.0,0.0,1.0,0.0,0.0,0.0
11,10001082,275221686,1.0,4.0,3.0,30.0,0.0,0.0,0.0
12,10001082,97441652,0.0,4.0,0.0,1.0,0.0,0.0,0.0


In [28]:
df_user_test.head()

,user_id,item_id,buy_18,count_user_buy,count_item_bought,count_look,count_favor,count_addcart,count_addcart_17
0,10001082,285259775,0.0,4.0,0.0,1.0,0.0,0.0,0.0
1,10001082,4368907,0.0,4.0,0.0,2.0,0.0,0.0,0.0
2,10001082,53616768,0.0,4.0,2.0,5.0,0.0,0.0,0.0
3,10001082,151466952,0.0,4.0,0.0,3.0,0.0,0.0,0.0
4,10001082,290088061,0.0,4.0,0.0,2.0,0.0,0.0,0.0


In [29]:
df_user_submit.head()

,user_id,item_id,count_user_buy,count_item_bought,count_look,count_favor,count_addcart,count_addcart_18
0,10001082,285259775,4.0,0.0,1.0,0.0,0.0,0.0
1,10001082,4368907,4.0,0.0,2.0,0.0,0.0,0.0
2,10001082,53616768,4.0,2.0,5.0,0.0,0.0,0.0
3,10001082,151466952,4.0,0.0,3.0,0.0,0.0,0.0
4,10001082,290088061,4.0,0.0,2.0,0.0,0.0,0.0


In [30]:
df_user_train.to_csv('./df_user_train.csv', index=False)

In [31]:
df_user_test.to_csv('./df_user_test.csv', index=False)

In [32]:
df_user_submit.to_csv('./df_user_submit.csv', index=False)